# A Tale of Two Hardware Sorters

There was a recent [Hackaday article](http://hackaday.com/2016/01/20/a-linear-time-sorting-algorithm-for-fpgas/) on a *hardware sorter* that could accept an unsorted list of numbers and return that list in sorted order.
This is a subject near and dear to my heart since I built a sorter chip in my first VLSI design class back in 1983.
So I thought I'd compare the Hackaday implementation with mine as follows:

* Code each sorter in [MyHDL](http://www.myhdl.org/) and simulate its operation.
* Build each sorter with an FPGA to gauge how much hardware it takes.

## Preliminary Setup

Before implementing each sorter, I need to setup my environment.

First, I'll need the `myhdl` package that augments Python with the capability to simulate digital hardware.

In [1]:
from myhdl import *

Next, I'll define the size of the numbers that the sorters will handle.
(In this case, the numbers will be *unsigned*, but that's easily changed by making `HS_MIN` a negative number.)

In [2]:
HS_WIDTH = 16
HS_MIN = 0
HS_MAX = 2**HS_WIDTH

Each sorter will be built as a *stack* of *cells* that will respond to the following set of commands:

* `RESET`: Reset all the cells in the stack to `HS_MIN`.
* `PUSH`: Enter a new number into the stack.
* `POP`: Remove a number from the stack.
* `NOP`: Don't do anything; the stack contents remain unchanged.

In [3]:
Cmd = enum('RESET', 'PUSH', 'POP', 'NOP', encoding='binary')

I'll also define a Python list that will be used to show the contents of the stack as the sorting proceeds.
(This is only used during the simulation and doesn't have any effect on the implementation in the FPGA.)

In [4]:
stack = []

OK, setup is done. Now let's build some sorters!

## The Hackaday Hardware Sorter

The operation of the Hackaday hardware sorter is simple:

* The sorter is implemented as a stack of cells, each one storing a single number. An unsorted list of numbers is pushed into the stack, and then a sorted list is read out by popping the stack.
* At all times, the numbers in the stack are in order with the largest number near the top of the stack (TOS) and the smallest number deepest within the stack.
* When a number is pushed into the stack, it is globally broadcast to each cell. Any cell with a number that is less than the broadcast number will push its number to its neighboring cell on the right. Because the numbers in the stack are always sorted, all the cells that follow a pushed cell will also be pushed because their numbers are also less than the global number. The broadcast number is stored in the left-most pushed cell because that cell is now empty.
* When extracting the sorted list, the number in each cell moves leftward to the next cell just like a regular stack.

For those who like a more graphical representation of the sorter's operations, here's a pretty picture that shows the broadcast-push-insert-pop operations. Note how the stack contents are always in order.

![had_sorter_steps](had_sorter_steps.png)

### The Basic Sorting Cell

Building the Hackaday sorter is done by designing its basic sorting cell and then stacking them together.

Here's the I/O needed for the cell:

* `clk_i`: The global clock signal that synchronizes the operation of all the cells in the stack.
* `cmd_i`: The global bus that carries the current command to all the cells (PUSH, POP, RESET, NOP).
* `input_i`: The global bus that carries the number being pushed into the stack.
* `shft_i`: The input that tells this cell if the cell to the left is sending its number here.
* `left_i`: The input that accepts the number from the cell on the left.
* `left_o`: The output that sends the number from this cell to the cell on the right.
* `right_i`: The input that accepts the number from the cell on the right.
* `right_o`: The output that sends the number from this cell to the one on the right.
* `shft_o`: The output that indicates this cell is sending its number to the cell on the right.

![Hackaday Sorter Cell](had_sorter_cell.png)

The MyHDL code for the basic cell of the Hackaday sorter stack is shown below.
There are three code blocks that define the operation of the cell:

* A combinational logic block that compares the number stored in the cell with
  whatever value is broadcast on the global input bus.
* A sequential logic block that updates the cell's number on every rising
  clock edge based on the operation being performed (PUSH, POP or RESET).
* A final combinational logic block that always outputs the number in the cell
  to the cells on either side and also tells the cell on the right when
  data is being shifted over to it.
  
There is a final block of code that's only used during simulation. It copies the number from
a cell in the stack into the equivalent position in a global array. The global array is used
to observe the contents of the stack as the sorting proceeds.

In [5]:
def had_hs_cell(clk_i, cmd_i, input_i, shft_i, left_i, left_o, right_i, right_o, shft_o, index):
    '''Basic cell for Hackaday hardware sorter:
            clk_i: The global clock signal.
            cmd_i: The current command to be executed.
            input_i: The global bus with the number being pushed into the stack.
            shft_i: This input is true if the cell to the left is sending its number here.
            left_i: The input that accepts the number from the cell on the left.
            left_o: The output that sends the register contents from this cell to the one on the left.
            right_i: The input that accepts the number from the cell on the right.
            right_o: The output that sends the register contents from this cell to the one on the right.
            shft_o: This output is true if this cell is sending its register contents to the right.
            index: The position of this cell in the stack. (Used only during simulation.)
    '''

    a = Signal(intbv(0,HS_MIN,HS_MAX))  # This register holds the number in the cell.
    a_lt_input = Signal(bool(0))  # This signal is true if the register contents are less than the input number.
    
    # This combinational logic block continually compares the contents 
    # of the cell register to the number on the global input bus.
    @always_comb
    def compare_a_input():
        a_lt_input.next = (a < input_i)
    
    # This sequential logic block updates the value in the cell register
    # on the rising edge of the clock signal.
    @always_seq(clk_i.posedge, reset=None)
    def update_a():
        if cmd_i == Cmd.RESET:
            a.next = HS_MIN  # Upon reset, set the cell register to the minimum possible value.
        elif cmd_i == Cmd.PUSH:
            # Handle pushing of data into the cell stack.
            if shft_i:
                # If the cell to the left is outputing its value, then this cell has to take it.
                a.next = left_i
            elif a_lt_input:
                # If the global input number is greater than the number in this cell
                # but the cell to the left isn't outputing its number, then
                # load the global input number into this cell register.
                a.next = input_i
        elif cmd_i == Cmd.POP:
            # When popping data from the stack, always accept the number from the cell on the immediate right.
            a.next = right_i
    
    # This combinational logic block controls the outputs from this cell.
    @always_comb
    def output_l_r():
        # The register contents are always made available to the cells on the right and left.
        right_o.next = a
        left_o.next = a
        # Tell the next cell on the right that data is coming if the number in this cell 
        # is less than the number on the global bus.
        shft_o.next = a_lt_input or shft_i

    # This combinational block is only used during simulation. It places the number in the
    # cell register into the stack list at the index of this cell.
    if observe:
        global stack  # For showing the stack contents during simulation.

        @always_comb
        def observe_a_b():
            stack[index] = (int(a.val),)
            
    return instances()

### Stacking the Cells

The stack for the Hackaday sorter is built by concatenating the basic cells as shown below.

![Hackaday sorter stack connections](had_sorter_stack.png)

The signals that go in to and out of the stack are:

* `clk_i`: The global clock signal.
* `cmd_i`: The current command to be executed.
* `unsorted_i`: The input bus for entering unsorted data.
* `sorted_o`: The output bus for reading the sorted data.

The MyHDL code for connecting the basic sorting cells is as follows:

In [6]:
def had_hs(clk_i, cmd_i, unsorted_i, sorted_o, n_cells):
    '''Hackaday hardware sorter stack:
            clk_i: The global clock signal.
            cmd_i: The current command to be executed.
            unsorted_i: The input bus for entering unsorted data.
            sorted_o: The output bus for reading the sorted data.
            n_cells: The number of cells to include in the stack.
    '''
    
    # These are the buses that take data from the cell on the left to the one on the right.
    l2r = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(0,n_cells+1)]
    # These are the buses that take data from the cell on the right to the one on the left.
    r2l = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(0,n_cells+1)]
    
    # These signals tell the cell on the right when they must accept data from the cell on the left.
    shft = [Signal(bool(0)) for _ in range(0,n_cells+1)]
    
    # No cell exists before the left-most cell, so inactivate the left-most shift input.
    shft[0] = bool(0)
    l2r[0] = 0

    # No cell exists after the right-most cell, so shift in the minimal value from the right.
    r2l[n_cells] = HS_MIN

    # The sorted output from the stack comes from the output of the left-most cell.
    r2l[0] = sorted_o
    
    # Connect each cell to its left and right neighbors and to the global input bus.
    cells = []
    for i in range(0, n_cells):
        cell = had_hs_cell(clk_i, cmd_i, unsorted_i, shft[i], l2r[i], r2l[i],
                            r2l[i+1], l2r[i+1], shft[i+1], i)
        cells.append(cell)
    
    return cells

### The Simulation Testbench

Now that the sorter is assembled, I need something to drive it during simulation to see if it works correctly. The following *testbench* does that like so:

1. It first instantiates the hardware sorter module and connects signals to its I/O ports.
2. Then the sorter is reset.
3. The numbers from an unsorted list are pushed into the stack through a series of clock pulses.
   The number stored in each cell is printed after each clock pulse so I can watch the progress of the sorter.
4. The sorted list is popped from the stack in a similar manner.
5. The unsorted and sorted lists are printed for comparison.
6. The built-in Python `sort()` function is used to sort the original list of numbers and then compare them to the
   output from the hardware sorter to verify that it operated correctly.

In [7]:
def hs_tb(hdw_sorter, unsorted_list, n_cells):
    '''Hardware sorter testbench:
            hdw_sorter: The name of the hardware sorter to be tested.
            unsorted_list: An unsorted list of numbers.
            n_cells: The number of cells to include in the hardware sorter.
    '''
    
    # Declare some signals that interface to the hardware sorter.
    clk = Signal(bool(0))
    cmd = Signal(Cmd.RESET)
    in_bus, out_bus = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(2)]
    
    # Instantiate the hardware sorter with the connections declared above.
    hs_inst = hdw_sorter(clk, cmd, in_bus, out_bus, n_cells)

    # The interface signals are driven in this block.
    @instance
    def tb():
        
        # A subroutine for showing the contents of the sorter cell registers.
        def show_stack(dir_arrow):
            s = ''
            for cell in stack:
                s += ' {} ( '.format(dir_arrow)
                for reg in cell:
                    s += '{:>5d} '.format(reg)
                s += ')'
            return s
        
        # Initialize the global list for recording the cell register contents in the stack.
        global stack
        stack = [0] * n_cells
        
        # Reset the hardware sorter.
        print('\nRESET:')
        cmd.next = Cmd.RESET  # Set the command input.
        # Pulse the clock low and then high.
        clk.next = 0
        yield(delay(1))
        clk.next = 1
        yield(delay(1))
        # Print the state of the cell registers after reset.
        print '     ', show_stack('  ')
        
        # Push the unsorted numbers into the sorter.
        print('\nPUSH:')
        cmd.next = Cmd.PUSH  # Set the command input.
        for num in unsorted_list:
            # Print the state of the cell registers and the current input number.
            print '{:>5d}'.format(num), show_stack('=>')
            # Send in the next number from the unsorted list.
            in_bus.next = num
            # Pulse the clock.
            clk.next = 0
            yield(delay(1))
            clk.next = 1
            yield(delay(1))

        # Print the state of the cell registers after the unsorted list has been pushed in.
        print '     ', show_stack('  ')
            
        # Pop the sorted numbers out of the sorter.
        print('\nPOP:')
        sorted_list = []  # Clear the list that will hold the sorted numbers.
        cmd.next = Cmd.POP  # Set the command input.
        for _ in unsorted_list:
            # Grab the current output from the sorter.
            sorted_list.append(int(out_bus.val))
            # Pulse the clock.
            clk.next = 0
            yield(delay(1))
            clk.next = 1
            yield(delay(1))
            # Print the state of the cell registers and the current output number.
            print '{:>5d}'.format(sorted_list[-1]), show_stack('<=')
            
        # The testbench simulation is done at this point.
        # Now just display and verify the results.

        # A subroutine for printing lists of numbers.
        def print_list(l):
            for i, n in enumerate(l,1):
                print('{:>7d}'.format(n)),
                if i%10 == 0:
                    print('')
            print('\n')
 
        # Print the original unsorted list and the sorted list from the sorter.
        print('')
        print('Unsorted list:')
        print_list(unsorted_list)
        print('Sorted list:')
        print_list(sorted_list)
        
        # Use Python sort() function to sort the unsorted list and compare it to the
        # output list from the sorter.
        verify_list = unsorted_list[:]
        verify_list.sort()
        if sorted_list == verify_list[::-1]:
            print('SUCCESS: List is correctly sorted!')
        else:
            print('ERROR: list is not correctly sorted!')
            
    return instances()

### Simulation Results

Now I'll create a list of unsorted numbers:

In [8]:
import random # Python package for generating random numbers.
LIST_LEN = 8
unsorted_list = [random.randrange(HS_MIN, HS_MAX) for _ in range(LIST_LEN)]

Then I can pass the unsorted numbers to the testbench along with the Hackaday sorter
and see the results of the simulation:

In [9]:
observe = True
Simulation(hs_tb(had_hs, unsorted_list, LIST_LEN)).run()


RESET:
          (     0 )    (     0 )    (     0 )    (     0 )    (     0 )    (     0 )    (     0 )    (     0 )

PUSH:
51275  => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 )
 9085  => ( 51275 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 )
49041  => ( 51275 ) => (  9085 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 )
44215  => ( 51275 ) => ( 49041 ) => (  9085 ) => (     0 ) => (     0 ) => (     0 ) => (     0 ) => (     0 )
46660  => ( 51275 ) => ( 49041 ) => ( 44215 ) => (  9085 ) => (     0 ) => (     0 ) => (     0 ) => (     0 )
19190  => ( 51275 ) => ( 49041 ) => ( 46660 ) => ( 44215 ) => (  9085 ) => (     0 ) => (     0 ) => (     0 )
 8925  => ( 51275 ) => ( 49041 ) => ( 46660 ) => ( 44215 ) => ( 19190 ) => (  9085 ) => (     0 ) => (     0 )
28920  => ( 51275 ) => ( 49041 ) => ( 46660 ) => ( 44215 ) => ( 19190 ) => (  9085 ) => (  8925 )

<class 'myhdl.StopSimulation'>: No more events


0

From the printed results, you can see where new numbers are inserted into the stack during the PUSH phase, and that the stack contents are always in descending order.
Then, removing the sorted list is just a matter of repeatedly popping the stack during the POP phase.

### FPGA Implementation

After all the work of simulating the Hackaday sorter, creating the Verilog code for it is surprisingly easy:

In [10]:
# Turn off the section of code that's only used during simulation.
observe = False

# Create the connecting buses for the sorter.
clk = Signal(bool(0))
cmd = Signal(Cmd.RESET)
unsorted_bus, sorted_bus = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(2)]

# Create the Verilog code for the sorter.
LIST_LEN = 8
toVerilog(had_hs, clk, cmd, unsorted_bus, sorted_bus, LIST_LEN)

# Show the Verilog file for the implementation and store it in another
# Verilog file with the size of the list it can sort.
with open('had_hs.v') as fr:
    verilog_src = fr.read()
    print(verilog_src)
    fw = open('had_hs_{}.v'.format(LIST_LEN), 'w')
    fw.write(verilog_src)
    fw.close()

// File: had_hs.v
// Generated by MyHDL 1.0dev
// Date: Wed Feb 24 22:35:25 2016


`timescale 1ns/10ps

module had_hs (
    clk_i,
    cmd_i,
    unsorted_i,
    sorted_o
);
// Hackaday hardware sorter stack:
// clk_i: The global clock signal.
// cmd_i: The current command to be executed.
// unsorted_i: The input bus for entering unsorted data.
// sorted_o: The output bus for reading the sorted data.
// n_cells: The number of cells to include in the stack.

input clk_i;
input [1:0] cmd_i;
input [15:0] unsorted_i;
output [15:0] sorted_o;
wire [15:0] sorted_o;

reg [15:0] cell_a;
wire [15:0] cell_left_o;
wire [15:0] cell_left_i;
wire [15:0] cell_right_o;
wire cell_shft_o;
wire cell_a_lt_input;
wire cell_shft_i;
reg [15:0] cells_6_a;
wire [15:0] cells_6_left_o;
wire [15:0] cells_6_left_i;
wire cells_6_a_lt_input;
wire cells_6_shft_i;
reg [15:0] cells_5_a;
wire [15:0] cells_5_left_o;
wire [15:0] cells_5_left_i;
wire cells_5_a_lt_input;
wire cells_5_shft_i;
reg [15:0] cells_4_a;
wire [15:0]

** ToVerilogWarning: Signal is driven but not read: cell_right_o
** ToVerilogWarning: Signal is driven but not read: cell_shft_o


## The IBM Hardware Sorter

The hardware sorter I built back in 1983 was based on an [IBM design](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.83.5786&rep=rep1&type=pdf).
It's similar to the Hackaday sorter with a few differences:

* The sorter is implemented as a stack of cells with *two numbers* stored in each cell. An unsorted list of numbers is 
  pushed into the stack, and then a sorted list is read out by popping the stack.
* When a number is pushed into the stack, it is sent only to the left-most cell. That cell sends the smaller 
  of the two numbers it stores to the next cell on the right while storing the incoming value in the vacated spot. The cell on the right does the same procedure,
  and so on through the entire stack. The net effect is that the smaller numbers propagate rightward through the stack, but
  the stack doesn't necessarilty have all the numbers correctly sorted at any time.
* When extracting the sorted list, the left-most cell outputs the larger of its two numbers. That number is replaced with
  the larger of the two numbers from the next cell on the right. This process repeats throughout the entire stack with 
  the net effect that the larger numbers propagate leftward through the stack.

It's not immediately obvious that the procedure described above will result in a sorted list being popped from the stack.
You can get a better feel for how the IBM sorter works by looking at the simulation output below.

### The Basic Sorting Cell

The basic cell for the IBM sorter has I/O signals similar to the Hackaday cell minus the
global input number bus and the shift signals.

![IBM Sorter Cell](ibm_sorter_cell.png)

The MyHDL code for the basic cell of the IBM sorter stack is shown below.
There are three code blocks that define the operation of the cell:

* A combinational logic block that compares the two numbers stored in the cell.
* A sequential logic block that updates the two numbers on every rising
  clock edge based on the operation being performed (PUSH, POP or RESET).
* A final combinational logic block that always outputs the larger number in the cell
  to the left and the smaller number to the right.
  
As with the Hackaday sorter, there's a final block of code that's only used during simulation
to observe the contents of the stack as the sorting proceeds.

In [11]:
def ibm_hs_cell(clk_i, cmd_i, left_i, left_o, right_i, right_o, index):
    '''Basic cell for IBM hardware sorter:
            clk_i: The global clock signal.
            cmd_i: The current command to be executed.
            left_i: The input that accepts the number from the cell on the left.
            left_o: The output that sends a number from this cell to the one on the left.
            right_i: The input that accepts the number from the cell on the right.
            right_o: The output that sends a number from this cell to the one on the right.
            index: This is the position of this cell in the stack. (Used only during simulation.)
    '''
    
    global stack  # For showing the stack contents during simulation.
    
    a, b = [Signal(intbv(0, HS_MIN, HS_MAX)) for _ in range(2)]  # Two registers for storing numbers.
    a_lt_b = Signal(bool(0))  # True if number in register a is less than number in register b.
    
    # This combinational logic block continually compares the contents 
    # of cell register a to the contents of cell register b.
    @always_comb
    def compare_a_b():
        a_lt_b.next = (a < b)
    
    # This sequential logic block updates the value in the cell registers
    # on the rising edge of the clock signal.
    @always_seq(clk_i.posedge, reset=None)
    def update_a_b():
        if cmd_i == Cmd.RESET:
            # Upon reset, set the registers to their minimum possible values.
            a.next = HS_MIN
            b.next = HS_MIN
        elif cmd_i == Cmd.PUSH:
            # When pushing data into the cell, place the number entering from
            # the left into the register containing the smallest number since
            # that number will be sent to the cell on the right.
            if a_lt_b:
                a.next = left_i
            else:
                b.next = left_i
        elif cmd_i == Cmd.POP:
            # When popping data from the cell, place the number entering from
            # the right into the register containing the largest number since
            # that number will be sent to the cell on the left.
            if a_lt_b:
                b.next = right_i
            else:
                a.next = right_i

    # This combinational logic block continually outputs:
    #   1) the smaller of the a or b registers goes to the right,
    #   2) the larger of the a or b registers goes to the left.
    @always_comb
    def output_l_r():
        if a_lt_b:
            left_o.next = b
            right_o.next = a
        else:
            left_o.next = a
            right_o.next = b

    # This combinational block is only used during simulation. It places the numbers
    # in the cell registers into the stack list at the index of this cell.
    if observe:
        @always_comb
        def observe_a_b():
            stack[index] = (int(a.val),int(b.val))
            
    return instances()

### Stacking the Cells

The stack for the IBM sorter is built by concatenating the basic cells as shown below.

![IBM sorter stack connections](ibm_sorter_stack.png)

The I/O signals for the stack are the same as for the Hackaday sorter stack:

* `clk_i`: The global clock signal.
* `cmd_i`: The current command to be executed.
* `unsorted_i`: The input bus for entering unsorted data.
* `sorted_o`: The output bus for reading the sorted data.

The MyHDL code for connecting the basic sorting cells is as follows:

In [12]:
def ibm_hs(clk_i, cmd_i, unsorted_i, sorted_o, n_cells):
    '''IBM hardware sorter stack:
            clk_i: The global clock signal.
            cmd_i: The current command to be executed.
            unsorted_i: The input bus for entering unsorted data.
            sorted_o: The output bus for reading the sorted data.
            n_cells: The number of cells to include in the stack.
    '''

    # These are the buses that take data from the cell on the left to the one on the right.
    l2r = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(0,n_cells+1)]
    # These are the buses that take data from the cell on the right to the one on the left.
    r2l = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(0,n_cells+1)]

    # The unsorted input to the stack goes into the left-most cell.
    l2r[0] = unsorted_i

    # The unsorted output from the stack comes from the output of the left-most cell.
    r2l[0] = sorted_o
    
    # No cell exists after the right-most cell, so shift in the minimal value from the right.
    r2l[n_cells] = HS_MIN
    
    # Connect each cell to the buses of its left and right neighbors.
    cells = []
    for i in range(0, n_cells):
        cell = ibm_hs_cell(clk_i, cmd_i, l2r[i], r2l[i], r2l[i+1], l2r[i+1], i)
        cells.append(cell)
    
    return cells

### Simulation Results

Just as I did for the Hackaday sorter, I can pass the IBM sorter to the testbench along with the list of unsorted numbers and see what happens. (For the IBM sorter, the number of sorter cells is only half the size of the list since each cell stores two numbers.)

In [13]:
observe = True
Simulation(hs_tb(ibm_hs, unsorted_list, len(unsorted_list)/2)).run()


RESET:
          (     0     0 )    (     0     0 )    (     0     0 )    (     0     0 )

PUSH:
51275  => (     0     0 ) => (     0     0 ) => (     0     0 ) => (     0     0 )
 9085  => (     0 51275 ) => (     0     0 ) => (     0     0 ) => (     0     0 )
49041  => (  9085 51275 ) => (     0     0 ) => (     0     0 ) => (     0     0 )
44215  => ( 49041 51275 ) => (     0  9085 ) => (     0     0 ) => (     0     0 )
46660  => ( 44215 51275 ) => ( 49041  9085 ) => (     0     0 ) => (     0     0 )
19190  => ( 46660 51275 ) => ( 49041 44215 ) => (     0  9085 ) => (     0     0 )
 8925  => ( 19190 51275 ) => ( 49041 46660 ) => ( 44215  9085 ) => (     0     0 )
28920  => (  8925 51275 ) => ( 49041 19190 ) => ( 44215 46660 ) => (     0  9085 )
          ( 28920 51275 )    ( 49041  8925 )    ( 19190 46660 )    ( 44215  9085 )

POP:
51275  <= ( 28920 49041 ) <= ( 46660  8925 ) <= ( 19190 44215 ) <= (     0  9085 )
49041  <= ( 28920 46660 ) <= ( 44215  8925 ) <= ( 19190  9085 ) <=

<class 'myhdl.StopSimulation'>: No more events


0

By looking at the stack contents above, you can see the smaller of the two numbers in each cell is propagated rightward
during the PUSH phase, while the larger number moves leftward during the POP phase.
At no time does the stack contain a strictly sorted list (unless the original list of numbers was sorted to begin with).

### FPGA Implementation

Now I can create the Verilog code for the IBM sorter like so:

In [14]:
# Turn off the section of code that's only used during simulation.
observe = False

# Create the connecting buses for the sorter.
clk = Signal(bool(0))
cmd = Signal(Cmd.RESET)
unsorted_bus, sorted_bus = [Signal(intbv(0,HS_MIN,HS_MAX)) for _ in range(2)]

# Create the Verilog code for the sorter.
LIST_LEN = 4
toVerilog(ibm_hs, clk, cmd, unsorted_bus, sorted_bus, LIST_LEN/2)

# Show the Verilog file for the implementation and store it in another
# Verilog file with the size of the list it can sort.
with open('ibm_hs.v') as fr:
    verilog_src = fr.read()
    print(verilog_src)
    fw = open('ibm_hs_{}.v'.format(LIST_LEN), 'w')
    fw.write(verilog_src)
    fw.close()    

// File: ibm_hs.v
// Generated by MyHDL 1.0dev
// Date: Wed Feb 24 22:35:25 2016


`timescale 1ns/10ps

module ibm_hs (
    clk_i,
    cmd_i,
    unsorted_i,
    sorted_o
);
// IBM hardware sorter stack:
// clk_i: The global clock signal.
// cmd_i: The current command to be executed.
// unsorted_i: The input bus for entering unsorted data.
// sorted_o: The output bus for reading the sorted data.
// n_cells: The number of cells to include in the stack.

input clk_i;
input [1:0] cmd_i;
input [15:0] unsorted_i;
output [15:0] sorted_o;
reg [15:0] sorted_o;

reg [15:0] cell_a;
reg [15:0] cell_left_o;
wire cell_a_lt_b;
reg [15:0] cell_right_o;
reg [15:0] cell_b;
reg [15:0] cell_left_i;
reg [15:0] cells_0_a;
wire cells_0_a_lt_b;
reg [15:0] cells_0_b;





always @(posedge clk_i) begin: IBM_HS_CELLS_0_UPDATE_A_B
    case (cmd_i)
        2'b00: begin
            cells_0_a <= 0;
            cells_0_b <= 0;
        end
        2'b01: begin
            if (cells_0_a_lt_b) begin
                cel

## FPGA Resource Usage

In order to test the efficiency of each hardware sorter, I generated Verilog source files for several
list lengths and then compiled them for a Lattice ICE40HX8K FPGA with their iCEcube2 design tool
(version 2015.08.27744).

Here are the results for the Hackaday sorter:

List Size  | #LUTs | #DFFs | #Carrys  | $F_{max}$ (MHz)
----------:|------:|------:|---------:|---------------:
4          | 175   | 64    | 64       | 103            
8          | 388   | 128   | 128      | 92             
16         | 835   | 256   | 256      | 83             
32         | 1918  | 512   | 512      | 68             
64         | 3596  | 1024  | 1024     | 55             
112        | 5892  | 1792  | 1792     | 46             
116        | 6168  | 1856  | 1856     | 42             
124        | 6432  | 1984  | 1984     | 43             

And these are the results for the IBM sorter:

List Size  | #LUTs | #DFFs | #Carrys  | $F_{max}$ (MHz)
----------:|------:|------:|---------:|---------------:
4          | 135   | 64    | 32       | 126
8          | 397   | 128   | 64       | 115
16         | 921   | 256   | 128      | 104
32         | 1969  | 512   | 256      | 96
64         | 4065  | 1024  | 512      | 92
112        | 7209  | 1792  | 896      | 79
116        | --    | --    | --       | --
124        | --    | --    | --       | --

The Hackaday sorter can handle slightly larger lists than the IBM sorter even though
it requires more carry blocks (since it requires a comparator for every number while
the IBM sorter only needs a comparator for every two numbers).
However, the multiplexing between the registers in each IBM sorter cell appears to
consume more lookup tables (LUTs) and this overshadows the effect of the carry cells.

The IBM sorter has a higher $F_{max}$ than the Hackaday sorter, so it can sort a list faster.
However, both sorters show a decrease in $F_{max}$ as the list size grows.
Since the critical path in both routers shouldn't increase with the list size, the
decrease in $F_{max}$ may be related to routing delays incurred while packing the
logic into the LUTs.

## Is It Worth It?

Probably not.

Trying to increase the speed of a microprocessor by hanging a hardware sorter off its external
bus is futile.
The time spent doing the I/O to/from the sorter would swamp any savings in sorting time for
the small lists these sorters can handle.
But if you had a processor in an FPGA and there was room for it, a hardware sorter might provide
some advantage due to the faster internal bus speeds.
That is, if you needed to do *a lot* of sorting.

But what is "worth it" is the use of MyHDL to explore these hardware sorters.
In 1983, it took me three months to design the VLSI chip for the IBM sorter, and then I waited
another six months to get a prototype.
Here, it took me five hours to design, simulate and implement the same sorter in a $10 FPGA.
(And much of that time was spent trying to gather and nicely format the simulator output for this notebook.)

In addition, this notebook is an *active carrier* of these designs. While the current document may be a static image if you're looking at it from my Github repository,
you can interact with it and try different approaches with the hardware sorters if you have a [Python environment](https://www.continuum.io/downloads) on your computer.
Just install MyHDL using either `pip install myhdl` or `easy_install myhdl` and you're ready to go!